# Geocoder

In [2]:
####### Települések geokódolása #######
import geocoder
import pandas as pd
import time
import numpy as np
import json
import requests


In [3]:
def read_abt(path):
    """
    (Source) Excel beolvasása df-be, külön df-be csak az irsz-ek, amit ledistinctelünk, hogy ne legyen felesleges lekérdezés
    Return: distinct_tav: dinstinct irsz df, source: source df
    """
    source = pd.read_csv(path)
    
    distinct_tav = source[['irsz', 'munka_irsz']].copy()
    
    distinct_tav.drop_duplicates(inplace=True)
    
    return distinct_tav, source

In [18]:
def get_coords(df):

    country = 'Tanzania'
    
    region_list = df.capital.to_list()
    
    coord_dict = {}

    for region in region_list:
        print(region)

        try:

            insert_address = region +','+ country
            g = geocoder.here(insert_address,
                               app_id='9s40bPAzWV12JBwgCLnj',
                               app_code='qlG3zbGQVQ5IlMY0FEAiRQ')

            coord = str(g.json['lat']) + ',' + str(g.json['lng'])

            coord_dict[region] = coord

        except:

            try:

                insert_address = region + ','+ country

                g = geocoder.here(insert_address,
                               app_id='9s40bPAzWV12JBwgCLnj',
                               app_code='qlG3zbGQVQ5IlMY0FEAiRQ')

                coord = str(g.json['lat']) + ',' + str(g.json['lng']) 

                coord_dict[region] = coord

            except:
                coord_dict[region] = ''+''
                print('not_ok')

        #print(coord)

    df['coord'] = df.capital.map(coord_dict)

    return df    

In [48]:
def get_dist(df):
    
    df['tav'] = np.nan
    
    for index, row in df.iterrows():
        
        from_geo = row['wpt_coord']
        to_geo = row['capital_coord']
        url = 'https://route.api.here.com/routing/7.2/calculateroute.json?app_id={app_id}&app_code={app_code}&waypoint0=geo!{from_geo}&waypoint1=geo!{to_geo}&mode=fastest;car;traffic:disabled&departure_time={dep_time}'.format(app_id='9s40bPAzWV12JBwgCLnj', app_code='qlG3zbGQVQ5IlMY0FEAiRQ',from_geo=from_geo, to_geo=to_geo, dep_time='2019-04-16T08:00:0')

        resp = json.loads(requests.get(url).content)

        #Check if response
        if 'response' in resp:
            status = 'OK'
            distance = resp.get('response').get('route')[0].get('summary').get('distance')
            trafficTime = resp.get('response').get('route')[0].get('summary').get('trafficTime')
            baseTime = resp.get('response').get('route')[0].get('summary').get('baseTime')
            travelTime =  resp.get('response').get('route')[0].get('summary').get('travelTime')
        else:
            status = 'ERROR'
            distance = ''
            trafficTime = ''
            baseTime = ''
            travelTime = ''
        
        df.loc[index,'tav'] = distance
        print(distance)
        
    return df

In [9]:
# Read in regions data
regions = pd.read_csv('..\\data-repo\\raw\\scraped\\regions.csv')

In [11]:
regions

,Unnamed: 0,region,capital,pop_2012,pop_2018_proj
0,1,Tanzania,Dodoma,44928923,54199200
1,2,Arusha,Arusha,1694310,1999900
2,3,Dar es Salaam,Dar es Salaam,4364541,5147100
3,4,Dodoma,Dodoma,2083588,2493000
4,5,Geita,Geita,1739530,2239900
5,6,Iringa,Iringa,941238,1095200
6,7,Kagera [West Lake],Bukoba,2458023,3022000
7,8,Katavi,Mpanda,564604,738200
8,9,Kigoma,Kigoma,2127930,2616200
9,10,Kilimanjaro,Moshi,1640087,1864300


In [30]:
regions = get_coords(regions)

Dodoma
Arusha
Dar es Salaam
Dodoma
Geita
Iringa
Bukoba
Mpanda
Kigoma
Moshi
Lindi
Babati
Musoma
Mbeya
Morogoro
Mtwara
Mwanza
Njombe
Dar es Salaam
Sumbawanga
Songea
Shinyanga
Bariadi
Singida
Vwawa
not_ok
Tabora
Tanga
Zanzibar


In [32]:
# Fill not found values
regions.at[24, 'coord'] = '-9.106425,32.930540'

In [36]:
regions['region'].replace({'Kagera [West Lake]': 'Kagera', 'Pwani [Coast]': 'Pwani'}, inplace=True)

In [41]:
regions.rename(columns={'coord': 'capital_coord'}, inplace=True)

In [42]:
# Read in denorm data
denorm = pd.read_csv('..\\data-repo\\cleaned\\denormalized.csv')
# Select relevant cols
distances = denorm[['latitude', 'longitude', 'region', 'capital']].copy()
# Remove dups
distances.drop_duplicates(inplace=True)
# Join capital coords

distances = distances.merge(regions, on=('region','capital'), how='left')

In [46]:
distances['latitude'] = distances['latitude'].astype(str)
distances['longitude'] = distances['longitude'].astype(str)

distances['wpt_coord'] = distances[['latitude', 'longitude']].apply(lambda x: ','.join(x), axis=1)

In [47]:
distances

,latitude,longitude,region,capital,Unnamed: 0,pop_2012,pop_2018_proj,capital_coord,wpt_coord
0,-2.14746569,34.6987661,Mara,Musoma,13,1743830,2209100,"-1.87905,33.69917","-2.14746569,34.6987661"
1,-3.82132853,37.46066446,Manyara,Babati,12,1425131,1750900,"-4.20865,35.74925","-3.82132853,37.46066446"
2,-11.15529772,38.48616088,Mtwara,Mtwara,16,1270854,1424100,"-10.29222,40.17741","-11.15529772,38.48616088"
3,-1.82535885,31.13084671,Kagera,Bukoba,7,2458023,3022000,"-1.33636,31.81396","-1.82535885,31.13084671"
4,-4.76558728,39.1727956,Tanga,Tanga,27,2045205,2337100,"-5.08635,39.0972","-4.76558728,39.1727956"
...,...,...,...,...,...,...,...,...,...
54022,-3.25384746,37.16980689,Kilimanjaro,Moshi,10,1640087,1864300,"-3.34481,37.34004","-3.25384746,37.16980689"
54023,-9.0706288,35.24999126,Iringa,Iringa,6,941238,1095200,"-7.76924,35.68917","-9.0706288,35.24999126"
54024,-8.75043433,34.01708706,Mbeya,Mbeya,14,1708548,2070400,"-8.9108,33.45781","-8.75043433,34.01708706"
54025,-6.37857327,35.86131531,Dodoma,Dodoma,4,2083588,2493000,"-6.17351,35.7469","-6.37857327,35.86131531"


In [49]:
tav = get_dist(distances)

145200
286269
249370
141586
46296
15202
131050
20679
196385
38288
18968
73298
40386
147419
49178
209159
215569

204154
130849
48259
14657
109873
130954
44200
39953
71441
107747
28397

14531
108705
190108
188774
21737
188019
42530
23156
24661
173010
39096
238793
4680
90931
106501
142740
2449
20180
6676
80402
64381
39827
159122
8149
38084
78285
50192
217416
171733
196863
51361
396358
144936
141811
140996
136155
4306
233591
40986
104157
198983
56581
9056
45552
87485
140254
94931
108464
39121
19410
108723
59818
53684
64216
116881
109383
72979
27858
33009
19110
115323
241098
308282
29321
47917
75917
255364
64424
32261
27160
270792
68265
112813
48975
27007
60841
103397
304856
41672
198352
107361
84761
88385
47714
237063
84888
27235
218017
27430
57560
135966
161480
151890
65038
230648
217881
105345
19352
49398
65887
39470
15666
46962
15804
124861
133676
43438
43044
139502
56810
277947
10845
162756
122359
51007
176668
104755
180187
106066
34864
82721
110139
160728
170040
75147
36419
130884
219

81336
20869
143614
255608
271917
59350
55192
64452
34619
284203
304715
179389
101075
12780
35788
126021
63963
116363
149703
103475
94274
230105
85133
36430
129987
33019
44012
20257
93568
184991
13519
86226
270530
187232
161383
62491
83153
97555
95387
37809
65840
49877
292233
74763
27429
61522
31676
198463
15379
117062
79463
58580
275388
139943
208282
59453
45814
19666
77319
157682
84499
37965
249832
35555
22552
45341
281580
135711
41854
133397
29135
80266
13775
16614
358694
147901
149844
73762
11380
68407
98251
23452
131992
60373
144736

20096
37792
132211
12922
2269
258114
150004
84406
158762
161460
12521
104781
121603
313241
96382
150422
159773
51478
18515
15880
387526
74602
173845
280241
203273
219189
202958
10034
220998
8183
106013
6261
117207
98157
41552
27531
98660
207054
20606
81272
153013
231192
221458
194406
138737
85206
103282
22143
150580
228127
20011
80427
60906
13281
90741
93898
47105
101803
107921
16331
47665
37790
48132
26388
362072
14718
98516
177594
256209
131813
15932

56460
213847
367056
40296
84481
17249
157599
30560
16825
152270
29804
2685
24257
91933
202993
93159
103825
22803

30144
82542
129919
182714
183747
299363
58908
119662
28501
44569
209315
107118
63346
146652
121618
169859
60580
227837
31874
102524
118772
154255
85588
191058
96095
36840
200512
212789
52752
13525
210864
14844
121204
39900
168768
158850
126792
69752
80409
169663
157753
39552
43438
17573
117955
16565
213352
55426
36554
11696
19821
134838
49859
136610
202282
322460
242772
266978
18276
91829
80628
138205
75539
13676
40234
163350
56604
231253
54124
23317
47639
9828
41873
94061
21376
51796
112809
153669
113482
60741
30743
106336
243030
200151
160086
52061
109438
199037
210708
214032
174361
142204
19946
28218
37812
54104
174091
159906
62505
158800
141553
221983
48605
51342
65978
173021
48854
95929
20569
24379
184851
210832
27560
221308
91519
122485
66468
29573
251781
117102
68533
66080
180977
182124
10670
72007
98539
180922
175814
12228
108261
148304
23696
85438
232521
177796
667

217429
151310
77236
227764
121082
49939
246285
50046
97880
78258
269667
231697
44799
153380
98406
84062
25488
267181
125963
44673
28998
61263
108426
44363
118002
70373
81622
62951
173238
24600
224110
13980
41059
176170
357275
94862
72520
188198
135784
199947
102519
96906
80882
111032
56916
33646
74257
26094
66522
320294
71937
183106
82467
132101
10751
97831
136369
81008
87122
36704
28594
33065
194194
40348
105345
31276
134874
84283
15401
33359
96253
23358
42012
106988
173488
181444
143236
103457
26644
243342
71482
20608
102603
65327
82200
70098
189167
28738
121981
315557
80833
79818
126570
41960
263509
4167
73086
17547
39609
226182
48051
14864
40569
93367
91076
50793
15502
150088
214874
87303
60642
43711
222083
3239
13332
282613
33937
10188
53501
266739

10472
85778
41233
67259
20578
40464
36843
107613
14284
88968
3525
82898
90907
29971
132088
132860
234127
47074
19004
172449
169216
41825
235359
38866
270666
59226
109655
61134
33450
82721
128747
35833
214768
27825
106538
97042
53487
26

132250
177882
31597
137935
14601
28299
38599
98952
14876
94095
74574
107317
178586
267475
32719
65449
78373
157820
164780
27437
171717
69830
52860
56593
99377
154093
404267
33762
33246
141100
206131
142778
54911
31969
12770
187540
84961
8546
292556
153130
70600
126477
199717
22220
233959
152806
28555
97818
174386
83880
253409
65578
39706
191557
24919
42878
89356
45744
180028
99710
97641
34746
124015
109446
42494
121249
66793
90912
25997
71817
189888
139820
43552
97718
69840
299183
3415
38963
25554
127291
158822
199615
116887
179756
221647
90873

127716
130400
376723
106313
29772
89649
83614
95523
197983
130976
65604
48221
116602
19782
31263
32164
10199
123208
22717
45622
17598
56752
21063
46485
66614
57071
28421
277851
47843
377485
146049
81202
145019
113479
189409
133048
112096
325459
299779
401435
271918
253776
28068
85829
130765
47491
36572
267133
28495
69716
85343
86977
93038
362706
119250
217871
48907
224677
157634
65746
61039
36828
76281
152696
279055
54968
245276
123162
25037
40

...

46122
80152
133912
41823
450
155435
225012
90482
94897
15856
175082
42865
106329
204138
154866
9435
197365
183113
86668
143378
103663
152825
66057
91328
70060
38947
26342
71179
279747
46334
102993
58209
156784
102100
125514
44822
32277
75381
233841
40680
254218
122175
45915
71515
178181
35115
121436
138155
122893
47925
99726
127448
91726
270295
261515
40620
97616
79564
84935
24514
113433
65768
9482
10732
170840
104479
111334
158417
3118
150937
81962
193298
178888
47686
118801
63292
21279
179487
111126
59311
142054
131492
4331
28581
85206
222010
109354
51905
60532
49348
110898
10355
25171
105144
90835
95800
19530
210665
188706
159078
273885
21876
26006
127210
84162
240080
141636
172539
153486
125567
64797
129475
75851
165522
221889
82081
180421
96960
388208
24014
365295
48604
54022
318978
174767
191296
142446
14071
197311
181796
16769
15301
99710
210178
146019
5597
220416
227408
79270
19262
90333
124279
17490
80010
207092

182599
127840
136923
36901
35410
65076
9367
170570
107903
107611

146166
26711
84727
108955
221200
102367
53222
51500
94376
65275

71970
156171
206538
90899
49453
133220
7749
35520
201282
65245
68064
118104
117621
126854
9452
93509
66052
91730
114484
74191
191159
51279
54083
101847
300805
58181
51732
82319
137714
99455
14005
179882
36537
128489
41390
30052
139131
67320
162543
11386
177500
153397
176037
46750
154201
129491
98748
178445
49950
161767
134667
199105
90127
158077
167285
15203
34162
64080
151506
110991
133104
15101
64709
1591
153992
142212
156133
201565
60011
58722
216824
60842
68458
111435
143376
18957
236360
30497
110175
67387
57168
335730
37590
165891
362583
65010
18078
108034
114008
207815
101453
46011
100377
280241
104051
15868
56456
5062
131354
95539
148449
46940
70683
380903
251470
8587
117449
2380
80371
388011
74830
227193
31566
107139
70496
54579
9238
141352
288115
37553
271224
77766
48189
34674
64613
179683
111112
15639
55660
8538
289864
12392
98014
63830
17656
59638
277431
119357
55026
341353
41894

180232
27115
68680
37706
26767

209533
74239
74969
58108
128000
54160
4470
241991
58540
37198
279372
97533
160707
91108
49032
51672
15138
304109
237096
92154
353801
91358
116968
43304
46543
144363
122777
341183
371855
147232
58577
281335
88131
96205
72224
20533
26084
25692
155842

128142
87121
53897
115320
151997
48091
27368
82266
110326
149184
123410
156453
143564
372290
147536
28680
94945
92764
225978
143927
15746
48296
75270
224510
120021
26412
98264
28914
266872
100605
9611
67934
89809
159760
7553
215317
138087
22065
95897
23740
247018
49814
80710
59446
65389
232543
123976
34160
100076
22917
284133
210366
99355
122617
21393
91265
74460
89036
140143
203190
174334
19499
20620
313744
71405
48813
57857
23472
15906
139547
382889
37870
106481
21552
83692
218334
36397
105083
55869
178476
55147
28562
160894
69624
82391
234195
135979
113023
103573
205483
93964
84309
133419
233819
69468
112151
66122
75472
93075
39961
57492
27324
72514
100325
23045
134080
72643
22852
70115
107271
62741
121428
108202
66712
281266
222988
1214

28844
180334
25047
144329
49404
94364

167335
12081
162115
20674
119300
245492
36076
83870
186349
86508
153377
85992
17204
96879
75511
70936
11636
114257
39584
84191
93329
132889
65580
111499
57564
135147
139468
140968
278585
42410
11869
368776
123110
106413
128036
163129
236160
172318
92631
76391
47775
65873
90576
226233
146169
51392
96734
105384
16192
69706
16807
96687
400619
83089
101396
46603
72721
19111
28860
94710
23264
125473
194547
124315
46296
103935

223539
10816
111075
23768
149830
131177
107637
249229
21562
62418
92251
172316
43643
148406
253430
175868
75276
222683
69477
8332
26485
84522
134756
38195
25729
38341
118494
194709
3785
146269
245313
113421
242772
32059
151164
209722
112454
59296
132596
35053
98836
37587
270177
206420
74345
86467
18339
80736
225946
45118
71728
62236
222487
183024
179714
50170
15205
39057
107739
18056
83085
63595
31862
152500
167844
119664
224659
132731
117816
57523
123493
63844
242399
176642
139794
218990
62698
56601
168228
23129
75367
48389
2222

227110
153797
50221
155899
80500
172348
85263
28559
223697
83772
31135
58947


In [51]:
tav.to_csv('..\\data-repo\\cleaned\\distances.csv')

In [3]:
pwd

'C:\\Users\\gerol\\Desktop\\CEU\\trim1\\DataArchitecture\\Assignment\\geo'

In [ ]:
cd

#### ABT-re

In [24]:
path = r'G:\Saját meghajtó\Common\Projekt\EON\20190502_Kormenedzsment\Adatok\ABT\ABT1205.csv'

tav, source = read_abt(path)

In [25]:
tav

,irsz,munka_irsz
0,4002.0,4030.0
4,2800.0,1051.0
5,2800.0,1134.0
8,7634.0,7636.0
12,9019.0,9027.0
...,...,...
28383,7477.0,7636.0
28411,4641.0,4400.0
28415,3915.0,4400.0
28419,1157.0,8000.0


In [26]:
tav = get_coords(tav)

irsz
4002.0
2800.0
2800.0
7634.0
9019.0
nan
0.0
9024.0
9700.0
9153.0
4002.0
1138.0
1138.0
7632.0
7668.0
8617.0
8617.0
8623.0
7081.0
9024.0
9027.0
9011.0
8229.0
8200.0
4100.0
8882.0
9012.0
2051.0
2051.0
7695.0
4800.0
4200.0
7400.0
0.0
9023.0
8454.0
1141.0
1141.0
1221.0
1013.0
9021.0
7627.0
8060.0
7636.0
7636.0
2941.0
5000.0
2009.0
1139.0
1139.0
1063.0
1063.0
4336.0
2700.0
2700.0
9352.0
4026.0
2030.0
4225.0
2461.0
7629.0
7628.0
4034.0
7400.0
7635.0
5430.0
2840.0
8600.0
8360.0
4080.0
4080.0
2899.0
8000.0
9231.0
4130.0
4130.0
1133.0
1133.0
1033.0
1033.0
1031.0
9025.0
8800.0
2112.0
2112.0
1076.0
1076.0
1171.0
4030.0
2030.0
1181.0
1183.0
0.0
7636.0
7632.0
1065.0
1144.0
2120.0
2120.0
1022.0
1022.0
1028.0
1028.0
7743.0
8960.0
8800.0
9029.0
1021.0
1021.0
1118.0
1118.0
2162.0
8854.0
1112.0
1116.0
1126.0
1116.0
8542.0
8043.0
5200.0
5200.0
4031.0
9791.0
4515.0
8500.0
8300.0
8300.0
2500.0
4461.0
7090.0
8360.0
8572.0
7900.0
4400.0
4232.0
4026.0
4211.0
4028.0
4032.0
4600.0
4621.0
5300.0
4287.0
4283.0

4080.0
8194.0
2220.0
7741.0
2092.0
4547.0
4547.0
8420.0
8621.0
1037.0
9485.0
4075.0
7020.0
8896.0
9025.0
7132.0
8881.0
4803.0
4803.0
4803.0
9330.0
4119.0
1066.0
1066.0
2500.0
2500.0
1048.0
1048.0
7333.0
7347.0
6726.0
3571.0
7171.0
4077.0
7633.0
4263.0
4556.0
4465.0
1074.0
7631.0
7400.0
3526.0
8296.0
2481.0
9171.0
8420.0
8092.0
1142.0
9985.0
1016.0
7305.0
8572.0
8283.0
9763.0
8852.0
1125.0
8852.0
5130.0
4556.0
8100.0
8983.0
4811.0
4132.0
8692.0
7634.0
1031.0
1188.0
1188.0
1153.0
1192.0
7471.0
8771.0
7476.0
7747.0
7747.0
7833.0
6449.0
0.0
9153.0
1029.0
3300.0
8790.0
1152.0
1157.0
6922.0
4100.0
4254.0
8130.0
9643.0
7304.0
8981.0
7211.0
9600.0
6500.0
0.0
8637.0
7355.0
0.0
9173.0
9361.0
8891.0
7100.0
4133.0
6800.0
4532.0
2433.0
8354.0
8414.0
7187.0
4803.0
4803.0
7682.0
4935.0
7187.0
8181.0
4434.0
9766.0
4932.0
8137.0
9721.0
9422.0
7121.0
7911.0
8879.0
8000.0
9012.0
9012.0
8799.0
7131.0
9342.0
2881.0
1045.0
1045.0
7629.0
9071.0
7054.0
7682.0
8800.0
4242.0
7143.0
7144.0
5561.0
5083.0
4266.0
4

1134.0
1051.0
1134.0
1051.0
1134.0
7636.0
8900.0
9027.0
9027.0
1051.0
1134.0
1051.0
1134.0
1134.0
8800.0
1134.0
1051.0
1134.0
1134.0
8500.0
8000.0
5081.0
5000.0
4030.0
9700.0
4400.0
8200.0
8200.0
8360.0
2800.0
4400.0
8600.0
8800.0
7400.0
7400.0
4400.0
4030.0
4030.0
4030.0
4030.0
4030.0
4400.0
4400.0
5300.0
4024.0
4030.0
4024.0
4024.0
4024.0
4024.0
4030.0
4024.0
5300.0
4400.0
4400.0
1051.0
4024.0
4400.0
1051.0
1134.0
4030.0
9027.0
9027.0
8800.0
7100.0
7626.0
7636.0
7626.0
8360.0
8800.0
7400.0
8360.0
8800.0
7636.0
7636.0
7100.0
2400.0
7636.0
2400.0
7636.0
7622.0
7636.0
7626.0
7636.0
7636.0
7100.0
7100.0
7100.0
7400.0
8800.0
8800.0
7626.0
8000.0
7100.0
2800.0
8000.0
7636.0
7626.0
7626.0
7626.0
9027.0
9027.0
4024.0
7636.0
7636.0
4100.0
4030.0
4030.0
4400.0
4700.0
4400.0
5000.0
8000.0
8200.0
9330.0
8000.0
2400.0
8360.0
8800.0
7636.0
7622.0
7100.0
9027.0
9700.0
4400.0
7636.0
8600.0
4400.0
8000.0
9330.0
9330.0
9027.0
2800.0
8900.0
8200.0
8200.0
8800.0
8800.0
7100.0
8360.0
7636.0
8200.0
9700.0

9027.0
2400.0
9700.0
7100.0
9024.0
9024.0
9330.0
8800.0
1134.0
4100.0
6800.0
4400.0
8000.0
9700.0
8200.0
7100.0
4400.0
4700.0
7626.0
4700.0
7636.0
8200.0
4030.0
9700.0
4700.0
8600.0
9027.0
9027.0
7100.0
7626.0
8900.0
7100.0
1051.0
1134.0
8800.0
7100.0
8200.0
9027.0
1051.0
1134.0
7621.0
7636.0
7100.0
7636.0
7636.0
4030.0
7130.0
7130.0
5000.0
5081.0
4024.0
4400.0
4400.0
4400.0
5000.0
5000.0
4100.0
4030.0
7622.0
7100.0
7400.0
8360.0
9027.0
4100.0
5300.0
5000.0
9027.0
8900.0
9700.0
2800.0
2400.0
7636.0
5300.0
4700.0
2800.0
8000.0
8000.0
8200.0
9700.0
2800.0
5000.0
2800.0
8900.0
8900.0
8000.0
8800.0
8360.0
2400.0
2400.0
9700.0
8200.0
8200.0
8800.0
4100.0
5300.0
2400.0
8000.0
8200.0
5000.0
4029.0
8200.0
8200.0
7300.0
4030.0
4024.0
9027.0
9027.0
9027.0
9700.0
8360.0
9027.0
9330.0
9700.0
2800.0
9700.0
1051.0
8360.0
9027.0
9027.0
8000.0
1051.0
1134.0
8000.0
4030.0
8000.0
8360.0
1051.0
4030.0
8360.0
4030.0
7636.0
4400.0
1051.0
1134.0
7100.0
8000.0
1051.0
9700.0
8000.0
9027.0
1051.0
8800.0
7636.0

In [27]:
tav

,irsz,munka_irsz,irsz_coord,munka_irsz_coord
0,4002.0,4030.0,"47.52894,21.50551","47.50712,21.63267"
4,2800.0,1051.0,"47.54806,18.42198","47.49923,19.05045"
5,2800.0,1134.0,"47.54806,18.42198","47.52402,19.06756"
8,7634.0,7636.0,"46.08051,18.17033","46.0221,18.20409"
12,9019.0,9027.0,"47.63405,17.59398","47.7001,17.68976"
...,...,...,...,...
28383,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409"
28411,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389"
28415,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389"
28419,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411"


In [29]:
tav.to_csv(r'G:\Saját meghajtó\Common\Projekt\EON\20190502_Kormenedzsment\Adatok\ABT_check\extensions\tav.csv')

In [36]:
tav = pd.read_csv(r'G:\Saját meghajtó\Common\Projekt\EON\20190502_Kormenedzsment\Adatok\ABT_check\extensions\tav.csv',
                 usecols = [1,2,3,4,5])

In [43]:
tav

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav
0,4002.0,4030.0,"47.52894,21.50551","47.50712,21.63267",12981.0
1,2800.0,1051.0,"47.54806,18.42198","47.49923,19.05045",58668.0
2,2800.0,1134.0,"47.54806,18.42198","47.52402,19.06756",62243.0
3,7634.0,7636.0,"46.08051,18.17033","46.0221,18.20409",8933.0
4,9019.0,9027.0,"47.63405,17.59398","47.7001,17.68976",18141.0
...,...,...,...,...,...
2247,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409",NaN
2248,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389",NaN
2249,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389",NaN
2250,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411",NaN


In [40]:
tav.loc[tav.tav.isna()]

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav
5,NaN,NaN,NaN,NaN,NaN
6,0.0,0.0,NaN,NaN,NaN
14,7668.0,7636.0,NaN,"46.0221,18.20409",NaN
29,7695.0,7636.0,NaN,"46.0221,18.20409",NaN
33,0.0,7400.0,NaN,"46.87181,18.56257",NaN
...,...,...,...,...,...
2247,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409",NaN
2248,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389",NaN
2249,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389",NaN
2250,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411",NaN


In [41]:
tav_test = tav.drop_duplicates(subset = ['irsz', 'munka_irsz'])

In [45]:
tav_test.drop('tav', axis = 1, inplace = True)

In [46]:
tav_test

,irsz,munka_irsz,irsz_coord,munka_irsz_coord
0,4002.0,4030.0,"47.52894,21.50551","47.50712,21.63267"
1,2800.0,1051.0,"47.54806,18.42198","47.49923,19.05045"
2,2800.0,1134.0,"47.54806,18.42198","47.52402,19.06756"
3,7634.0,7636.0,"46.08051,18.17033","46.0221,18.20409"
4,9019.0,9027.0,"47.63405,17.59398","47.7001,17.68976"
...,...,...,...,...
2247,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409"
2248,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389"
2249,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389"
2250,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411"


In [47]:
get_dist(tav)

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav
0,4002.0,4030.0,"47.52894,21.50551","47.50712,21.63267",12981
1,2800.0,1051.0,"47.54806,18.42198","47.49923,19.05045",58668
2,2800.0,1134.0,"47.54806,18.42198","47.52402,19.06756",62243
3,7634.0,7636.0,"46.08051,18.17033","46.0221,18.20409",8933
4,9019.0,9027.0,"47.63405,17.59398","47.7001,17.68976",18141
...,...,...,...,...,...
2247,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409",74932
2248,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389",64320
2249,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389",43048
2250,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411",74896


In [48]:
tav

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav
0,4002.0,4030.0,"47.52894,21.50551","47.50712,21.63267",12981
1,2800.0,1051.0,"47.54806,18.42198","47.49923,19.05045",58668
2,2800.0,1134.0,"47.54806,18.42198","47.52402,19.06756",62243
3,7634.0,7636.0,"46.08051,18.17033","46.0221,18.20409",8933
4,9019.0,9027.0,"47.63405,17.59398","47.7001,17.68976",18141
...,...,...,...,...,...
2247,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409",74932
2248,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389",64320
2249,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389",43048
2250,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411",74896


In [49]:
tav.to_csv(r'G:\Saját meghajtó\Common\Projekt\EON\20190502_Kormenedzsment\Adatok\ABT_check\extensions\tav_dist.csv')

### Get tavolsag for missing irsz and munka irsz combinations

In [102]:
tav = pd.read_csv(r'G:\Saját meghajtó\Common\Projekt\EON\20190502_Kormenedzsment\Adatok\ABT_check\extensions\tav_missing.csv')


In [103]:
tav = tav[['irsz', 'munka_irsz']].copy()

In [104]:
tav.drop_duplicates(inplace=True)

In [105]:
tav.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 168
Data columns (total 2 columns):
irsz          29 non-null float64
munka_irsz    29 non-null float64
dtypes: float64(2)
memory usage: 696.0 bytes


In [294]:
distances = pd.read_csv(r'G:\Saját meghajtó\Common\Projekt\EON\20190502_Kormenedzsment\Adatok\ABT_check\extensions\tav_dist.csv',
                       usecols = [1,2,3,4,5])


In [295]:
distances

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav
0,4002.0,4030.0,"47.52894,21.50551","47.50712,21.63267",12981.0
1,2800.0,1051.0,"47.54806,18.42198","47.49923,19.05045",58668.0
2,2800.0,1134.0,"47.54806,18.42198","47.52402,19.06756",62243.0
3,7634.0,7636.0,"46.08051,18.17033","46.0221,18.20409",8933.0
4,9019.0,9027.0,"47.63405,17.59398","47.7001,17.68976",18141.0
...,...,...,...,...,...
2247,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409",74932.0
2248,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389",64320.0
2249,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389",43048.0
2250,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411",74896.0


In [296]:
distances = distances.merge(res2, on='irsz', how='left', suffixes=('', '_corr'))

In [297]:
def fix_coords(x):
    
    if type(x['irsz_coord']) != str:
        return x['irsz_coord_corr']
    else:
        return x['irsz_coord']

In [298]:
distances

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav,irsz_coord_corr
0,4002.0,4030.0,"47.52894,21.50551","47.50712,21.63267",12981.0,NaN
1,2800.0,1051.0,"47.54806,18.42198","47.49923,19.05045",58668.0,NaN
2,2800.0,1134.0,"47.54806,18.42198","47.52402,19.06756",62243.0,NaN
3,7634.0,7636.0,"46.08051,18.17033","46.0221,18.20409",8933.0,NaN
4,9019.0,9027.0,"47.63405,17.59398","47.7001,17.68976",18141.0,NaN
...,...,...,...,...,...,...
2247,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409",74932.0,NaN
2248,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389",64320.0,NaN
2249,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389",43048.0,NaN
2250,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411",74896.0,NaN


In [299]:
distances.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2252 entries, 0 to 2251
Data columns (total 6 columns):
irsz                2251 non-null float64
munka_irsz          2251 non-null float64
irsz_coord          2199 non-null object
munka_irsz_coord    2245 non-null object
tav                 2194 non-null float64
irsz_coord_corr     29 non-null object
dtypes: float64(3), object(3)
memory usage: 123.2+ KB


In [300]:
distances['irsz_coord'] = distances.apply(lambda x: fix_coords(x), axis=1)

In [301]:
distances.loc[distances['irsz_coord'].isna()]

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav,irsz_coord_corr
5,NaN,NaN,NaN,NaN,NaN,NaN
6,0.0,0.0,NaN,NaN,NaN,NaN
33,0.0,7400.0,NaN,"46.87181,18.56257",NaN,NaN
92,0.0,7636.0,NaN,"46.0221,18.20409",NaN,NaN
313,0.0,9027.0,NaN,"47.7001,17.68976",NaN,NaN
473,0.0,4030.0,NaN,"47.50712,21.63267",NaN,NaN
481,0.0,4024.0,NaN,"47.52846,21.63105",NaN,NaN
507,0.0,2800.0,NaN,"47.54806,18.42198",NaN,NaN
510,0.0,9700.0,NaN,"47.21434,16.6243",NaN,NaN
523,0.0,6721.0,NaN,"46.25845,20.15361",NaN,NaN


In [302]:
distances_todo = distances.loc[distances['irsz'] != 0]

In [303]:
distances_todo = distances.loc[~((distances['irsz'].isna()) | (distances['irsz'] == 0 ))]
distances_todo= distances_todo.loc[distances_todo['tav'].isna()]

distances_todo = distances_todo.drop('tav', axis=1)
distances_todo

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,irsz_coord_corr
14,7668.0,7636.0,"46.00563,18.18179","46.0221,18.20409","46.00563,18.18179"
29,7695.0,7636.0,"46.23703,18.47998","46.0221,18.20409","46.23703,18.47998"
165,7668.0,7626.0,"46.00563,18.18179","46.07864,18.24179","46.00563,18.18179"
411,9064.0,9027.0,"47.670134, 17.629755","47.7001,17.68976","47.670134, 17.629755"
446,4002.0,0.0,"47.52894,21.50551",NaN,NaN
491,7728.0,0.0,"45.98332,18.6833",NaN,NaN
503,7671.0,7636.0,"46.01706,18.09974","46.0221,18.20409","46.01706,18.09974"
518,9061.0,9027.0,"47.76644,17.62996","47.7001,17.68976","47.76644,17.62996"
595,7664.0,7636.0,"46.08602,18.40121","46.0221,18.20409","46.08602,18.40121"
723,7633.0,0.0,"46.06238,18.19995",NaN,NaN


In [220]:
distances_finished = get_dist(distances_todo)

In [221]:
distances_finished

,irsz,irsz_coord,munka_irsz_coord,irsz_coord_corr,tav
14,7668.0,"46.00563,18.18179","46.0221,18.20409","46.00563,18.18179",5098
29,7695.0,"46.23703,18.47998","46.0221,18.20409","46.23703,18.47998",41911
165,7668.0,"46.00563,18.18179","46.07864,18.24179","46.00563,18.18179",10660
411,9064.0,"47.670134, 17.629755","47.7001,17.68976","47.670134, 17.629755",
446,4002.0,"47.52894,21.50551",NaN,NaN,
491,7728.0,"45.98332,18.6833",NaN,NaN,
503,7671.0,"46.01706,18.09974","46.0221,18.20409","46.01706,18.09974",13060
518,9061.0,"47.76644,17.62996","47.7001,17.68976","47.76644,17.62996",15691
595,7664.0,"46.08602,18.40121","46.0221,18.20409","46.08602,18.40121",28760
723,7633.0,"46.06238,18.19995",NaN,NaN,


In [222]:
distances

,irsz,irsz_coord,munka_irsz_coord,tav,irsz_coord_corr
0,4002.0,"47.52894,21.50551","47.50712,21.63267",12981.0,NaN
1,2800.0,"47.54806,18.42198","47.49923,19.05045",58668.0,NaN
2,2800.0,"47.54806,18.42198","47.52402,19.06756",62243.0,NaN
3,7634.0,"46.08051,18.17033","46.0221,18.20409",8933.0,NaN
4,9019.0,"47.63405,17.59398","47.7001,17.68976",18141.0,NaN
...,...,...,...,...,...
2247,7477.0,"46.37862,17.81615","46.0221,18.20409",74932.0,NaN
2248,4641.0,"48.27366,22.22042","47.95271,21.67389",64320.0,NaN
2249,3915.0,"48.11669,21.3366","47.95271,21.67389",43048.0,NaN
2250,1157.0,"47.53952,19.15232","47.19458,18.43411",74896.0,NaN


In [304]:
distances = distances.merge(distances_finished[['irsz_coord','munka_irsz_coord','tav']], 
                                on = ('irsz_coord','munka_irsz_coord'), suffixes=('','_corr'),
                               how='left')

In [305]:
distances

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav,irsz_coord_corr,tav_corr
0,4002.0,4030.0,"47.52894,21.50551","47.50712,21.63267",12981.0,NaN,NaN
1,2800.0,1051.0,"47.54806,18.42198","47.49923,19.05045",58668.0,NaN,NaN
2,2800.0,1134.0,"47.54806,18.42198","47.52402,19.06756",62243.0,NaN,NaN
3,7634.0,7636.0,"46.08051,18.17033","46.0221,18.20409",8933.0,NaN,NaN
4,9019.0,9027.0,"47.63405,17.59398","47.7001,17.68976",18141.0,NaN,NaN
...,...,...,...,...,...,...,...
2249,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409",74932.0,NaN,NaN
2250,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389",64320.0,NaN,NaN
2251,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389",43048.0,NaN,NaN
2252,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411",74896.0,NaN,NaN


In [306]:
def fix_tav(x):
    
    if np.isnan(x['tav']) and x['tav_corr'] != "":
        return x['tav_corr']
    else:
        return x['tav']

In [307]:
distances.loc[distances_try['tav'].isna()]

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav,irsz_coord_corr,tav_corr
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.0,0.0,NaN,NaN,NaN,NaN,NaN
33,0.0,7400.0,NaN,"46.87181,18.56257",NaN,NaN,NaN
92,0.0,7636.0,NaN,"46.0221,18.20409",NaN,NaN,NaN
313,0.0,9027.0,NaN,"47.7001,17.68976",NaN,NaN,NaN
411,9064.0,9027.0,"47.670134, 17.629755","47.7001,17.68976",NaN,"47.670134, 17.629755",
446,4002.0,0.0,"47.52894,21.50551",NaN,NaN,NaN,
473,0.0,4030.0,NaN,"47.50712,21.63267",NaN,NaN,NaN
481,0.0,4024.0,NaN,"47.52846,21.63105",NaN,NaN,NaN
491,7728.0,0.0,"45.98332,18.6833",NaN,NaN,NaN,


In [308]:
distances['tav'] = distances.apply(lambda x: fix_tav(x), axis=1)

In [309]:
tav_miss = distances.loc[distances['tav'].isna()]
tav_miss

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav,irsz_coord_corr,tav_corr
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.0,0.0,NaN,NaN,NaN,NaN,NaN
33,0.0,7400.0,NaN,"46.87181,18.56257",NaN,NaN,NaN
92,0.0,7636.0,NaN,"46.0221,18.20409",NaN,NaN,NaN
313,0.0,9027.0,NaN,"47.7001,17.68976",NaN,NaN,NaN
411,9064.0,9027.0,"47.670134, 17.629755","47.7001,17.68976",NaN,"47.670134, 17.629755",
446,4002.0,0.0,"47.52894,21.50551",NaN,NaN,NaN,
473,0.0,4030.0,NaN,"47.50712,21.63267",NaN,NaN,NaN
481,0.0,4024.0,NaN,"47.52846,21.63105",NaN,NaN,NaN
491,7728.0,0.0,"45.98332,18.6833",NaN,NaN,NaN,


In [311]:
tav_miss = tav_miss.loc[(~tav_miss['irsz_coord'].isna()) & (~tav_miss['munka_irsz_coord'].isna())].copy()

In [312]:
tav_miss

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav,irsz_coord_corr,tav_corr
411,9064.0,9027.0,"47.670134, 17.629755","47.7001,17.68976",NaN,"47.670134, 17.629755",
829,9004.0,9027.0,"47.528527, 21.776511","47.7001,17.68976",NaN,"47.528527, 21.776511",
1210,4077.0,4030.0,"47.680044, 17.647354","47.50712,21.63267",NaN,"47.680044, 17.647354",
1675,9051.0,9027.0,"47.728836, 17.660242","47.7001,17.68976",NaN,"47.728836, 17.660242",
1676,9051.0,9027.0,"47.728836, 17.660242","47.7001,17.68976",NaN,"47.728836, 17.660242",
1807,1059.0,1051.0,"47.500158, 19.050059","47.49923,19.05045",NaN,"47.500158, 19.050059",
1808,1059.0,1134.0,"47.500158, 19.050059","47.52402,19.06756",NaN,"47.500158, 19.050059",
2089,4077.0,4024.0,"47.680044, 17.647354","47.52846,21.63105",NaN,"47.680044, 17.647354",
2193,9041.0,9027.0,"47.728836, 17.660242","47.7001,17.68976",NaN,"47.728836, 17.660242",
2194,9041.0,9027.0,"47.728836, 17.660242","47.7001,17.68976",NaN,"47.728836, 17.660242",


In [313]:
tav_miss = tav_miss[['irsz_coord', 'munka_irsz_coord']].copy()

In [314]:
tav_miss.drop_duplicates(inplace=True)

In [282]:
tav_miss.to_excel(r'G:\Saját meghajtó\Common\Projekt\EON\20190502_Kormenedzsment\Adatok\ABT_check\extensions\man_calc.xlsx')

In [277]:
tav_miss.shape

(7, 2)

In [278]:
manual_calc = [7400, 353000, 343000, 8400, 260, 3300, 344000]
len(manual_calc)

7

In [315]:
tav_miss['tav'] = manual_calc

In [316]:
tav_miss

,irsz_coord,munka_irsz_coord,tav
411,"47.670134, 17.629755","47.7001,17.68976",7400
829,"47.528527, 21.776511","47.7001,17.68976",353000
1210,"47.680044, 17.647354","47.50712,21.63267",343000
1675,"47.728836, 17.660242","47.7001,17.68976",8400
1807,"47.500158, 19.050059","47.49923,19.05045",260
1808,"47.500158, 19.050059","47.52402,19.06756",3300
2089,"47.680044, 17.647354","47.52846,21.63105",344000


In [317]:
distances.drop(['irsz_coord_corr', 'tav_corr'], inplace=True, axis=1)

In [318]:
distances = distances.merge(tav_miss[['irsz_coord','munka_irsz_coord','tav']], 
                                on = ('irsz_coord','munka_irsz_coord'), suffixes=('','_corr'),
                               how='left')

In [319]:
distances

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav,tav_corr
0,4002.0,4030.0,"47.52894,21.50551","47.50712,21.63267",12981.0,NaN
1,2800.0,1051.0,"47.54806,18.42198","47.49923,19.05045",58668.0,NaN
2,2800.0,1134.0,"47.54806,18.42198","47.52402,19.06756",62243.0,NaN
3,7634.0,7636.0,"46.08051,18.17033","46.0221,18.20409",8933.0,NaN
4,9019.0,9027.0,"47.63405,17.59398","47.7001,17.68976",18141.0,NaN
...,...,...,...,...,...,...
2249,7477.0,7636.0,"46.37862,17.81615","46.0221,18.20409",74932.0,NaN
2250,4641.0,4400.0,"48.27366,22.22042","47.95271,21.67389",64320.0,NaN
2251,3915.0,4400.0,"48.11669,21.3366","47.95271,21.67389",43048.0,NaN
2252,1157.0,8000.0,"47.53952,19.15232","47.19458,18.43411",74896.0,NaN


In [320]:
distances['tav'] = distances.apply(lambda x: fix_tav(x), axis=1)

In [321]:
distances.loc[distances['tav'].isna()]

,irsz,munka_irsz,irsz_coord,munka_irsz_coord,tav,tav_corr
5,NaN,NaN,NaN,NaN,NaN,NaN
6,0.0,0.0,NaN,NaN,NaN,NaN
33,0.0,7400.0,NaN,"46.87181,18.56257",NaN,NaN
92,0.0,7636.0,NaN,"46.0221,18.20409",NaN,NaN
313,0.0,9027.0,NaN,"47.7001,17.68976",NaN,NaN
446,4002.0,0.0,"47.52894,21.50551",NaN,NaN,NaN
473,0.0,4030.0,NaN,"47.50712,21.63267",NaN,NaN
481,0.0,4024.0,NaN,"47.52846,21.63105",NaN,NaN
491,7728.0,0.0,"45.98332,18.6833",NaN,NaN,NaN
507,0.0,2800.0,NaN,"47.54806,18.42198",NaN,NaN


In [322]:
distances.to_csv(r'G:\Saját meghajtó\Common\Projekt\EON\20190502_Kormenedzsment\Adatok\ABT_check\extensions\tav_dist1209.csv',
                i)

In [325]:
distances.to_excel(r'G:\Saját meghajtó\Common\Projekt\EON\20190502_Kormenedzsment\Adatok\ABT_check\extensions\tav_dist1209.xlsx',
                  index=False)

In [251]:
myres

,irsz_coord,munka_irsz_coord,tav
411,"47.670134, 17.629755","47.7001,17.68976",
829,"47.528527, 21.776511","47.7001,17.68976",
1210,"47.680044, 17.647354","47.50712,21.63267",
1675,"47.728836, 17.660242","47.7001,17.68976",
1676,"47.728836, 17.660242","47.7001,17.68976",
1807,"47.500158, 19.050059","47.49923,19.05045",
1808,"47.500158, 19.050059","47.52402,19.06756",
2089,"47.680044, 17.647354","47.52846,21.63105",
2193,"47.728836, 17.660242","47.7001,17.68976",
2194,"47.728836, 17.660242","47.7001,17.68976",
